In [1]:
import json
import os
import pandas as pd
import numpy as np
import tweepy
import csv
from os import listdir
import datetime as dt

for each row that fits our criteria, inserts it to the DF

In [2]:
def populate_tweet_df(tweets):
    df = pd.DataFrame()
    user_lang = list(map(lambda tweet: tweet['user']['lang'], tweets))
    #is_retweet = list(map(lambda tweet: tweet['retweeted_status'], tweets)) #boolean

    #if is_retweet == 'False':
    df['TweetID'] = list(map(lambda tweet: tweet['id'], tweets))
    df['UserID'] = list(map(lambda tweet: tweet['user']['id'], tweets))
    df['UserName'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets))
    df['Text'] = list(map(lambda tweet: tweet['text'], tweets))
    df['Location'] = list(map(lambda tweet: tweet['user']['location'], tweets))
    df['User Language'] = list(map(lambda tweet: tweet['user']['lang'], tweets))
    df['Tweet Language'] = list(map(lambda tweet: tweet['lang'], tweets))
    df['Date'] = list(map(lambda tweet: tweet['created_at'], tweets))
    return df

# JSON files
this code takes all json files in a directory and converts it to single DF

In [5]:
directory = r'C:\Users\matoo\Documents\4th year\final project\some code\Test\\'
tweet_files = os.listdir(directory)
tweets = []
languages =['bg','cs','fr','ru','es','nl','de','it','ko','ja']
for file in tweet_files:
    with open(directory+file, 'r') as f:
        for line in f.readlines():
            try:
                jsonObject = json.loads(line)
                lang = jsonObject['user']['lang']
                #isRetweet = jsonObject.get('retweeted_status')
                if any(lang in s for s in languages)and 'retweeted_status' not in jsonObject:
                    tweets.append(json.loads(line))
            except ValueError:
                print "error in file: ", file, "\nline: ", line
        df = populate_tweet_df(tweets)

Remove URLs, mentions (@) and hashtags (#).
We keep spaces, remove punctuation and no big letters (all toLower)

In [6]:
#remove URLs
df.Text = (df.Text).str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
                                '', case=False)
#remove menstions
df.Text = (df.Text).str.replace('\@\w+', '', case=False)
#remove hashtags
df.Text = (df.Text).str.replace('\#\w+', '', case=False)
#remove special signs
df.Text = (df.Text).str.replace(r'\n', ' ', case=False)
df.Text = (df.Text).str.replace(r'\r', ' ', case=False)
#remove punctuation
df.Text = (df.Text).str.replace(r"([^a-zA-Z'0-9 ])", ' ', case=False) #replace with space
#ToLower:
df.Text = (df.Text).str.lower()

In [7]:
print df.shape
print df.head(5)

(144113, 8)
              TweetID              UserID         UserName  \
0  864251041621192705           561988281        ADOTRADIO   
1  864251041620967425  728487742200549378      auto_Chekov   
2  864251041436622849          1662294480       halifaxca1   
3  864251041344311296           125054226  AlexKamillaKroy   
4  864251041184960514           385454278       hurricvnne   

                                                Text  \
0     now playing  side are you on   bianco  appl...   
1  ugh    zhey're keep moving  i can't get a lock...   
2  warranty administrator   steele subaru   halif...   
3  fbi   me at jfk terminal  publically say u hac...   
4  tell them i was happy and my heart is broken a...   

                    Location User Language Tweet Language  \
0                                       it             en   
1  USS Enterprise (NCC-1701)            ko             en   
2             Halifax Canada            nl             en   
3             St. Petersburg      

# pickle files
if we need to add old pickles

In [8]:
languages =['bg','cs','fr','ru','es','nl','de','it','ko','ja']

old_df_path = r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\corpus_to_pickle_all_langs_new4'
old_df = pd.read_pickle(old_df_path)
#cols = df.columns
#dfBG = pd.read_pickle(r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\BGtweets')
#dfKO = pd.read_pickle(r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\KOtweets')
#dfCS = pd.read_pickle(r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\CStweets')
#
#dfBG.columns = cols
#dfKO.columns = cols
#dfCS.columns = cols

frames = [df, old_df]#, dfKO, dfBG, dfCS]
df = pd.concat(frames)

df = df[df['User Language'].isin(languages)]

df = df.drop_duplicates(subset ='TweetID', keep='last')

check the DF dimensions

In [9]:
print df.shape
print df.head(5)

(956110, 8)
              TweetID              UserID         UserName  \
0  864251041621192705           561988281        ADOTRADIO   
1  864251041620967425  728487742200549378      auto_Chekov   
2  864251041436622849          1662294480       halifaxca1   
3  864251041344311296           125054226  AlexKamillaKroy   
4  864251041184960514           385454278       hurricvnne   

                                                Text  \
0     now playing  side are you on   bianco  appl...   
1  ugh    zhey're keep moving  i can't get a lock...   
2  warranty administrator   steele subaru   halif...   
3  fbi   me at jfk terminal  publically say u hac...   
4  tell them i was happy and my heart is broken a...   

                    Location User Language Tweet Language  \
0                                       it             en   
1  USS Enterprise (NCC-1701)            ko             en   
2             Halifax Canada            nl             en   
3             St. Petersburg      

converting our DF to pickle file

In [10]:
df.to_pickle(r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\corpus_to_pickle_all_langs_final')

If we want to check what is in an existing pickle

In [16]:
df = pd.read_pickle(r'C:\Users\matoo\Documents\4th year\final project\some code\Pickles\final_pickle_no_duplicats')

some stats on the corpus

In [17]:
num_of_tweets = df.copy(deep = True)
total_lang = pd.DataFrame(num_of_tweets.groupby('User Language').size().reset_index())
total_users = pd.DataFrame(num_of_tweets.groupby('User Language').UserID.nunique().reset_index())
print "total languages: "
print total_lang
print "total users: "
print total_users

total languages: 
  User Language       0
0            bg    9868
1            cs   24262
2            de   84479
3            es  246201
4            fr  124844
5            it   47325
6            ja   35152
7            ko   32464
8            nl   50691
9            ru   80205
total users: 
  User Language  UserID
0            bg     520
1            cs    1707
2            de   41315
3            es  141251
4            fr   67992
5            it   24118
6            ja   21167
7            ko    5623
8            nl   22757
9            ru   23403


In [18]:
aggregate = {'UserID':{'total_tweets':'count'}}
total_users_in_langs = pd.DataFrame(num_of_tweets.groupby(['UserID','User Language']).agg(aggregate).reset_index())
total_users_in_langs.columns = ['UserID', 'User Language','total_tweets']
print total_users_in_langs.head()
aggregated_features = {'total_tweets':['min','max','var','mean','median']}
stats_per_lang = pd.DataFrame(total_users_in_langs.groupby('User Language').agg(aggregated_features))
print stats_per_lang

   UserID User Language  total_tweets
0    5199            fr             1
1    5430            de             1
2   10781            nl             2
3   53883            it             1
4   55293            es             1
              total_tweets                                    
                       min  max          var       mean median
User Language                                                 
bg                       1  187  1583.698888  18.976923      1
cs                       1  298  1066.607492  14.213240      1
de                       1  265    18.199261   2.044754      1
es                       1  401    19.892982   1.743004      1
fr                       1  299    18.272724   1.836157      1
it                       1  373    21.594750   1.962227      1
ja                       1  363    17.201414   1.660698      1
ko                       1  267   290.382311   5.773431      1
nl                       1  327    28.656938   2.227490      1
ru              

Output all stats to excel. May take a while to write.

In [19]:
writer = pd.ExcelWriter('Statistics_final_no dups.xlsx')
total_lang.to_excel(writer,'Total languages')
total_users.to_excel(writer,'Users per language')
stats_per_lang.to_excel(writer,'Language users stats')
total_users_in_langs.to_excel(writer,'All users per language')
writer.save()

just for testing

In [56]:
print num_of_tweets.loc[df['UserID'] == 725658846367244288]

                  TweetID              UserID     UserName  \
23003  839246187689562112  725658846367244288  MiafKhalifa   
34969  840297081533280260  725658846367244288  MiafKhalifa   
43747  840659371063463937  725658846367244288  MiafKhalifa   
52017  841021841221267456  725658846367244288  MiafKhalifa   
54699  840913236035801088  725658846367244288  MiafKhalifa   
57409  841420478145273858  725658846367244288  MiafKhalifa   
64116  841782797245247490  725658846367244288  MiafKhalifa   
64128  841782795630440448  725658846367244288  MiafKhalifa   
95252  842761290628186112  725658846367244288  MiafKhalifa   

                                                    Text Location  \
23003                          you cheat on your girl?         US   
34969   To Convince Your Girlfriend To Do The Naughti...       US   
43747                            You Mean A Lot To Him         US   
52017               Between Making Love And Having Sex         US   
54699   She Saw Her Son Sexist Pos